In [235]:
from sklearn import preprocessing
from time import time
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import csv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_diabetes
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import silhouette_samples, silhouette_score
from operator import truediv
from sklearn.metrics import pairwise_distances
import pandas as pd
get_ipython().magic('matplotlib')

df=pd.read_csv('C:/diabetic_data_processed_withweight.csv',';')
to_del = ['encounter_id', 'patient_nbr','medical_specialty','payer_code']
print (to_del)
#Filter_selected cols
filtered_cols = [c for c in df.columns if (c not in to_del) ]#and ('ENF' not in c)
df_2 = df[filtered_cols]
print ("df_2",df_2.shape)
print(df_2.columns)

Using matplotlib backend: Qt5Agg
['encounter_id', 'patient_nbr', 'medical_specialty', 'payer_code']
df_2 (101766, 46)
Index(['race', 'gender', 'age', 'weight', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')


In [256]:
for i in range (0,df_2['diag_3'].size):
    if ((df_2.loc[i,'diag_1']==410 or df_2.loc[i,'diag_1']==412) or (df_2.loc[i,'diag_2']==410 or df_2.loc[i,'diag_2']==412) or (df_2.loc[i,'diag_3']==410 or df_2.loc[i,'diag_3']==412)):
        #print(df_2['diag_1'],df_2['diag_2'],df_2['diag_3'])
        df_2.loc[i,'Myocardial_infarction']=1
    else:
        df_2.loc[i,'Myocardial_infarction']=0

In [218]:
print(df_2['Myocardial_infarction'].astype('category').value_counts())
listA=[]
listA1=[]
listA=df_2.loc[:,'Myocardial_infarction']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Myocardial_infarction']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['A1Cresult'].describe())
mean=df_2['A1Cresult'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'A1Cresult']> mean
listD1=[]
listD=df_2.loc[:,'A1Cresult']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listMI=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Myocardial_infarction']==1):
        listMI.append(df_2.loc[x,:])    
df_MI = pd.DataFrame(listMI)
df_MI['age'].describe()

import scipy as sp
import scipy.stats
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0*np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h
print("Age",mean_confidence_interval(df_MI['age']))
print("Gender",df_MI['gender'].astype('category').value_counts())
print("Race",df_MI['race'].astype('category').value_counts())
print("Readmitted",df_MI['readmitted'].astype('category').value_counts())

0.0    96985
1.0     4781
Name: Myocardial_infarction, dtype: int64
count    101766.000000
mean          0.242537
std           3.124083
min          -1.000000
25%          -1.000000
50%          -1.000000
75%          -1.000000
max           9.000000
Name: A1Cresult, dtype: float64
mean 0.2425368001100564
17018 4781 4781 84748
1028 1028 3753 3753
Weight--> RR:  1.0 , RD:  0.0 , OR:  0.2739142019717559
Age (74.080736247646939, 73.703162351840874, 74.458310143453005)
Gender 0.0    2645
1.0    2136
Name: gender, dtype: int64
Race 0.0    3937
1.0     533
2.0      98
3.0      68
4.0      24
Name: race, dtype: int64
Readmitted 0.0     2856
31.0    1432
29.0     493
Name: readmitted, dtype: int64


In [257]:
for i in range (0,df_2['diag_3'].size):
    if (df_2.loc[i,'diag_1']==428 or df_2.loc[i,'diag_2']==428 or (df_2.loc[i,'diag_3']==428)):
        #print(df_2['diag_1'],df_2['diag_2'],df_2['diag_3'])
        df_2.loc[i,'Congestive_heart_failure']=1
    else:
        df_2.loc[i,'Congestive_heart_failure']=0      

In [220]:
print(df_2['Congestive_heart_failure'].astype('category').value_counts())
listA=[]
listA1=[]
listA=df_2.loc[:,'Congestive_heart_failure']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Congestive_heart_failure']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['weight'].describe())
mean=df_2['weight'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'weight']> mean
listD1=[]
listD=df_2.loc[:,'weight']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","RR: ",RR,", RD: ",RD,", OR: ",OR)

listA=[]
listA1=[]
listA=df_2.loc[:,'Congestive_heart_failure']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Congestive_heart_failure']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['A1Cresult'].describe())
mean=df_2['A1Cresult'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'A1Cresult']> mean
listD1=[]
listD=df_2.loc[:,'A1Cresult']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("A1CResult-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listCHF=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Congestive_heart_failure']==1):
        listCHF.append(df_2.loc[x,:])    
df_CHF = pd.DataFrame(listCHF)
df_CHF['age'].describe()

print("Age",mean_confidence_interval(df_CHF['age']))
print("Gender",df_CHF['gender'].astype('category').value_counts())
print("Race",df_CHF['race'].astype('category').value_counts())
print("Readmitted",df_CHF['readmitted'].astype('category').value_counts())

0    84302
1    17464
Name: Congestive_heart_failure, dtype: int64


C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


count    3197.000000
mean       92.595246
std        25.465480
min        25.000000
25%              NaN
50%              NaN
75%              NaN
max       201.000000
Name: weight, dtype: float64
mean 92.59524554269628
819 17464 17464 2378
109 109 332 332
Weight--> RR:  1.0 , RD:  0.0 , OR:  0.32831325301204817
count    101766.000000
mean          0.242537
std           3.124083
min          -1.000000
25%          -1.000000
50%          -1.000000
75%          -1.000000
max           9.000000
Name: A1Cresult, dtype: float64
mean 0.2425368001100564
17018 17464 17464 84748
2712 2712 14752 14752
A1CResult--> RR:  1.0 , RD:  0.0 , OR:  0.18383947939262474
Age (77.187356848373796, 76.999992235787019, 77.374721460960572)
Gender 1.0    9547
0.0    7917
Name: gender, dtype: int64
Race 0.0    13273
1.0     3351
3.0      262
2.0      186
4.0       61
Name: race, dtype: int64
Readmitted 0.0     7809
31.0    7365
29.0    2290
Name: readmitted, dtype: int64


In [241]:
for i in range (0,df_2['diag_3'].size):
    if (df_2.loc[i,'diag_1'] in (443,441,785,43) 
        or df_2.loc[i,'diag_2'] in (443,441,785,43)  
        or df_2.loc[i,'diag_3'] in (443,441,785,43)):
        df_2.loc[i,'Peripheral_vascular_disease']=1
    else:
        df_2.loc[i,'Peripheral_vascular_disease']=0

C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [244]:
print(df_2['Peripheral_vascular_disease'].astype('category').value_counts())
listA=[]
listA1=[]
listA=df_2.loc[:,'Peripheral_vascular_disease']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Peripheral_vascular_disease']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['weight'].describe())
mean=df_2['weight'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'weight']> mean
listD1=[]
listD=df_2.loc[:,'weight']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","RR: ",RR,", RD: ",RD,", OR: ",OR)

listA=[]
listA1=[]
listA=df_2.loc[:,'Peripheral_vascular_disease']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Peripheral_vascular_disease']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['A1Cresult'].describe())
mean=df_2['A1Cresult'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'A1Cresult']> mean
listD1=[]
listD=df_2.loc[:,'A1Cresult']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("A1CResult-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listCHF=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Peripheral_vascular_disease']==1):
        listCHF.append(df_2.loc[x,:])    
df_CHF = pd.DataFrame(listCHF)
print(df_CHF['age'].describe())

print("Age",mean_confidence_interval(df_CHF['age']))
print("Gender",df_CHF['gender'].astype('category').value_counts())
print("Race",df_CHF['race'].astype('category').value_counts())
print("Readmitted",df_CHF['readmitted'].astype('category').value_counts())

0    99851
1     1915
Name: Peripheral_vascular_disease, dtype: int64


C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


count    3197.000000
mean       92.595246
std        25.465480
min        25.000000
25%              NaN
50%              NaN
75%              NaN
max       201.000000
Name: weight, dtype: float64
mean 92.59524554269628
819 1915 1915 2378
23 23 41 41
Weight--> RR:  1.0 , RD:  0.0 , OR:  0.5609756097560976
count    101766.000000
mean          0.242537
std           3.124083
min          -1.000000
25%          -1.000000
50%          -1.000000
75%          -1.000000
max           9.000000
Name: A1Cresult, dtype: float64
mean 0.2425368001100564
17018 1915 1915 84748
281 281 1634 1634
A1CResult--> RR:  1.0 , RD:  0.0 , OR:  0.17197062423500611
Age (72.44908616187989, 71.828490629770698, 73.069681693989082)
Gender 0.0    1005
1.0     910
Name: gender, dtype: int64
Race 0.0    1478
1.0     318
3.0      36
2.0      29
4.0       7
Name: race, dtype: int64
Readmitted 0.0     1040
31.0     619
29.0     256
Name: readmitted, dtype: int64


In [246]:
for i in range (0,df_2['diag_3'].size):
    if ((df_2.loc[i,'diag_1']>=430 and df_2.loc[i,'diag_1']<=438) or (df_2.loc[i,'diag_2']>=430 and df_2.loc[i,'diag_2']<=438) or (df_2.loc[i,'diag_3']>=430 and df_2.loc[i,'diag_3']<=438)):
        df_2.loc[i,'Cerebrovascular_disease']=1
    else:
        df_2.loc[i,'Cerebrovascular_disease']=0

C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [247]:
print(df_2['Cerebrovascular_disease'].astype('category').value_counts())
listA=[]
listA1=[]
listA=df_2.loc[:,'Cerebrovascular_disease']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Cerebrovascular_disease']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['weight'].describe())
mean=df_2['weight'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'weight']> mean
listD1=[]
listD=df_2.loc[:,'weight']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","RR: ",RR,", RD: ",RD,", OR: ",OR)

listA=[]
listA1=[]
listA=df_2.loc[:,'Cerebrovascular_disease']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Cerebrovascular_disease']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['A1Cresult'].describe())
mean=df_2['A1Cresult'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'A1Cresult']> mean
listD1=[]
listD=df_2.loc[:,'A1Cresult']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("A1CResult-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listCHF=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Cerebrovascular_disease']==1):
        listCHF.append(df_2.loc[x,:])    
df_CHF = pd.DataFrame(listCHF)
print(df_CHF['age'].describe())

print("Age",mean_confidence_interval(df_CHF['age']))
print("Gender",df_CHF['gender'].astype('category').value_counts())
print("Race",df_CHF['race'].astype('category').value_counts())
print("Readmitted",df_CHF['readmitted'].astype('category').value_counts())

0.0    96197
1.0     5569
Name: Cerebrovascular_disease, dtype: int64


C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


count    3197.000000
mean       92.595246
std        25.465480
min        25.000000
25%              NaN
50%              NaN
75%              NaN
max       201.000000
Name: weight, dtype: float64
mean 92.59524554269628
819 5569 5569 2378
27 27 130 130
Weight--> RR:  1.0 , RD:  0.0 , OR:  0.2076923076923077
count    101766.000000
mean          0.242537
std           3.124083
min          -1.000000
25%          -1.000000
50%          -1.000000
75%          -1.000000
max           9.000000
Name: A1Cresult, dtype: float64
mean 0.2425368001100564
17018 5569 5569 84748
1132 1132 4437 4437
A1CResult--> RR:  1.0 , RD:  0.0 , OR:  0.2551273382916385
count    5569.000000
mean       75.760460
std        12.326413
min        10.000000
25%        70.000000
50%        80.000000
75%        80.000000
max       100.000000
Name: age, dtype: float64
Age (75.760459687556107, 75.436649424379453, 76.084269950732761)
Gender 1.0    2961
0.0    2608
Name: gender, dtype: int64
Race 0.0    3971
1.0    1243
3.0 

In [ ]:
for i in range (0,df_2['diag_3'].size):
    if (df_2.loc[i,'diag_1']==290 or df_2.loc[i,'diag_2']==290 or df_2.loc[i,'diag_3']==290):
        df_2.loc[i,'Dementia']=1
    else:
        df_2.loc[i,'Dementia']=0

C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [252]:
print(df_2['Dementia'].astype('category').value_counts())
listA=[]
listA1=[]
listA=df_2.loc[:,'Dementia']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Dementia']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['weight'].describe())
mean=df_2['weight'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'weight']> mean
listD1=[]
listD=df_2.loc[:,'weight']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
#RR=(a/(a+b)) / (c/(c+d))
#RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","OR: ",OR)

listA=[]
listA1=[]
listA=df_2.loc[:,'Dementia']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Dementia']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['A1Cresult'].describe())
mean=df_2['A1Cresult'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'A1Cresult']> mean
listD1=[]
listD=df_2.loc[:,'A1Cresult']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("A1CResult-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listCHF=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Dementia']==1):
        listCHF.append(df_2.loc[x,:])    
df_CHF = pd.DataFrame(listCHF)
print(df_CHF['age'].describe())

print("Age",mean_confidence_interval(df_CHF['age']))
print("Gender",df_CHF['gender'].astype('category').value_counts())
print("Race",df_CHF['race'].astype('category').value_counts())
print("Readmitted",df_CHF['readmitted'].astype('category').value_counts())

0.0    101555
1.0       211
Name: Dementia, dtype: int64


C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


count    3197.000000
mean       92.595246
std        25.465480
min        25.000000
25%              NaN
50%              NaN
75%              NaN
max       201.000000
Name: weight, dtype: float64
mean 92.59524554269628
819 211 211 2378
0 0 4 4
Weight--> OR:  0.0
count    101766.000000
mean          0.242537
std           3.124083
min          -1.000000
25%          -1.000000
50%          -1.000000
75%          -1.000000
max           9.000000
Name: A1Cresult, dtype: float64
mean 0.2425368001100564
17018 211 211 84748
44 44 167 167
A1CResult--> RR:  1.0 , RD:  0.0 , OR:  0.2634730538922156
count    211.000000
mean      83.459716
std        9.898150
min       50.000000
25%       80.000000
50%       90.000000
75%       90.000000
max      100.000000
Name: age, dtype: float64
Age (83.459715639810426, 82.116421937900355, 84.803009341720497)
Gender 1.0    122
0.0     89
Name: gender, dtype: int64
Race 0.0    159
1.0     41
2.0      5
3.0      2
4.0      1
Name: race, dtype: int64
Readmitted 

In [253]:
for i in range (0,df_2['diag_3'].size):
    if ((df_2.loc[i,'diag_1']>=490 and df_2.loc[i,'diag_1']<=506) or (df_2.loc[i,'diag_2']>=490 and df_2.loc[i,'diag_2']<=506) or (df_2.loc[i,'diag_3']>=490 and df_2.loc[i,'diag_3']<=506)):
        df_2.loc[i,'Chronic_pulmonary_disease']=1
    else:
        df_2.loc[i,'Chronic_pulmonary_disease']=0

In [254]:
print(df_2['Chronic_pulmonary_disease'].astype('category').value_counts())
listA=[]
listA1=[]
listA=df_2.loc[:,'Chronic_pulmonary_disease']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Chronic_pulmonary_disease']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['weight'].describe())
mean=df_2['weight'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'weight']> mean
listD1=[]
listD=df_2.loc[:,'weight']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","RR: ",RR,", RD: ",RD,", OR: ",OR)

listA=[]
listA1=[]
listA=df_2.loc[:,'Chronic_pulmonary_disease']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Chronic_pulmonary_disease']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['A1Cresult'].describe())
mean=df_2['A1Cresult'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'A1Cresult']> mean
listD1=[]
listD=df_2.loc[:,'A1Cresult']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("A1CResult-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listCHF=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Chronic_pulmonary_disease']==1):
        listCHF.append(df_2.loc[x,:])    
df_CHF = pd.DataFrame(listCHF)
print(df_CHF['age'].describe())

print("Age",mean_confidence_interval(df_CHF['age']))
print("Gender",df_CHF['gender'].astype('category').value_counts())
print("Race",df_CHF['race'].astype('category').value_counts())
print("Readmitted",df_CHF['readmitted'].astype('category').value_counts())

0.0    88126
1.0    13640
Name: Chronic_pulmonary_disease, dtype: int64


C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


count    3197.000000
mean       92.595246
std        25.465480
min        25.000000
25%              NaN
50%              NaN
75%              NaN
max       201.000000
Name: weight, dtype: float64
mean 92.59524554269628
819 13640 13640 2378
92 92 233 233
Weight--> RR:  1.0 , RD:  0.0 , OR:  0.3948497854077253
count    101766.000000
mean          0.242537
std           3.124083
min          -1.000000
25%          -1.000000
50%          -1.000000
75%          -1.000000
max           9.000000
Name: A1Cresult, dtype: float64
mean 0.2425368001100564
17018 13640 13640 84748
2238 2238 11402 11402
A1CResult--> RR:  1.0 , RD:  0.0 , OR:  0.196281354148395
count    13640.000000
mean        73.488270
std         13.325404
min         10.000000
25%         70.000000
50%         70.000000
75%         80.000000
max        100.000000
Name: age, dtype: float64
Age (73.488269794721404, 73.264624460374989, 73.711915129067819)
Gender 1.0    7518
0.0    6122
Name: gender, dtype: int64
Race 0.0    11090
1.

In [269]:
print(df_2.columns)

Index(['race', 'gender', 'age', 'weight', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'Peripheral_vascular_disease', 'Cerebrovascular_disease', 'Dementia',
       'Chronic_pulmonary_disease', 'Myocardial_infarction',
       'Congestive_heart_fa

In [287]:
to_del2 = ['diag_1', 'diag_2','diag_3']
print (to_del2)
#Filter_selected cols
filtered_cols = [c for c in df_2.columns if (c not in to_del2) ]#and ('ENF' not in c)
df_3 = df_2[filtered_cols]
print ("df_3",df_3.shape)
print(df_3.columns)

['diag_1', 'diag_2', 'diag_3']
df_3 (101766, 49)
Index(['race', 'gender', 'age', 'weight', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'Peripheral_vascular_disease', 'Cerebrovascular_disease', 'Dementia',
       'Chronic_pulmonary_disease', 'Myocardial_infarction',
       '

In [288]:
mid = df_3['readmitted']
df_3.drop(labels=['readmitted'], axis=1,inplace = True)
df_3.insert(0, 'readmitted', mid)
print(df_3.shape)


msk = np.random.rand(len(df_3)) < 0.8
df, df_test = df_3[msk].copy(deep = True), df_3[~msk].copy(deep = True)
df = df.reset_index()
df_test = df_test.reset_index()
print("df.shape",df.shape)
print("df_test.shape",df_test.shape)
y_train=df['readmitted']
y_test=df_test['readmitted']
print(set(y_train))
print(set(y_test))

x_train=df.iloc[:,1:50]
x_test=df_test.iloc[:,1:50]
print(set(x_train))
print(set(x_test))

(101766, 49)
df.shape (81306, 50)
df_test.shape (20460, 50)
{0, 29, 31}
{0, 29, 31}
{'miglitol', 'num_lab_procedures', 'insulin', 'weight', 'metformin-rosiglitazone', 'repaglinide', 'rosiglitazone', 'nateglinide', 'age', 'discharge_disposition_id', 'num_procedures', 'A1Cresult', 'readmitted', 'number_diagnoses', 'time_in_hospital', 'Cerebrovascular_disease', 'tolbutamide', 'Dementia', 'number_emergency', 'examide', 'Congestive_heart_failure', 'acarbose', 'citoglipton', 'glyburide-metformin', 'max_glu_serum', 'num_medications', 'chlorpropamide', 'admission_type_id', 'Peripheral_vascular_disease', 'race', 'glyburide', 'metformin-pioglitazone', 'admission_source_id', 'Chronic_pulmonary_disease', 'number_inpatient', 'glimepiride', 'number_outpatient', 'metformin', 'gender', 'glipizide', 'troglitazone', 'glipizide-metformin', 'pioglitazone', 'acetohexamide', 'change', 'tolazamide', 'glimepiride-pioglitazone', 'diabetesMed', 'Myocardial_infarction'}
{'miglitol', 'num_lab_procedures', 'insuli

C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [289]:
print(df_3.columns)

Index(['readmitted', 'race', 'gender', 'age', 'weight', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed',
       'Peripheral_vascular_disease', 'Cerebrovascular_disease', 'Dementia',
       'Chronic_pulmonary_disease', 'Myocardial_infarction',
       'Congestive_heart_failure'],
      dtype='object')

In [350]:
#Fill na
df_4 = df_3.fillna(value=np.mean(df_3,axis=0),inplace=False,axis=0).values
print ("df_4",df_4.shape)

scaler = preprocessing.MinMaxScaler().fit(df_4)
data = scaler.transform(df_4)
print ("data",data.shape)

df_4 (101766, 49)
data (101766, 49)


In [351]:
#### Feat sel with pca(0.8, 0.9) explained var 
msk = np.random.rand(len(data)) < 0.8
newdata, newdata_test = data[msk].copy(), data[~msk].copy()

data=newdata

thrd = 0.8
total = 0
pca = PCA().fit(data)
reduced_data = pca.transform(data)
for pca_comps,r in enumerate(pca.explained_variance_ratio_):
    if total > thrd:
        break
    total += r
print ("Num pca_comps per >", thrd,"ratio:", pca_comps, total)
print ("Explained variance first 2 components",pca.explained_variance_ratio_[0]+pca.explained_variance_ratio_[1])
print (pca.n_components_)
print (np.sum(pca.explained_variance_ratio_[:2]))

print ("PCA+K-means:", pca_comps)

Num pca_comps per > 0.8 ratio: 11 0.808905325842
Explained variance first 2 components 0.339700396065
49
0.339700396065
PCA+K-means: 11


In [345]:
from sklearn.metrics import silhouette_samples, silhouette_score
for k in [2,3,4,5,6,7,8,9,10]:
    kmeans = KMeans(init='k-means++', n_clusters=k, n_init=10)
    cluster_labels = kmeans.fit_predict(reduced_data[:,:pca_comps])
    #cluster_labels=kmeans.fit(reduced_data[:,:2])
    #silhouette_avg = silhouette_score(reduced_data[:,:pca_comps], cluster_labels)
    #silhouette_avg = silhouette_score(reduced_data[:,:2], cluster_labels)
    #print("For n_clusters =", k, "The average silhouette_score is :", silhouette_avg)
    calinski_harabaz_score_avg = metrics.calinski_harabaz_score(reduced_data[:,:pca_comps], cluster_labels)
    #calinski_harabaz_score_avg = metrics.calinski_harabaz_score(reduced_data[:,:2], cluster_labels)
    print("For n_clusters =", k," the average metrics.calinski_harabaz_score is :", calinski_harabaz_score_avg)

For n_clusters = 2  the average metrics.calinski_harabaz_score is : 22650.8106865
For n_clusters = 3  the average metrics.calinski_harabaz_score is : 18818.0893855
For n_clusters = 4  the average metrics.calinski_harabaz_score is : 17973.1714991
For n_clusters = 5  the average metrics.calinski_harabaz_score is : 16869.4599806
For n_clusters = 6  the average metrics.calinski_harabaz_score is : 15269.6169899
For n_clusters = 7  the average metrics.calinski_harabaz_score is : 14867.5171957
For n_clusters = 8  the average metrics.calinski_harabaz_score is : 13801.2201756
For n_clusters = 9  the average metrics.calinski_harabaz_score is : 13417.937195
For n_clusters = 10  the average metrics.calinski_harabaz_score is : 13132.8078569


In [352]:
n_clusters = 2
pca = PCA().fit(data)
cluster_labels = kmeans.fit_predict(reduced_data[:,:2])
plt.figure()
plt.plot(range(len(pca.explained_variance_ratio_)), np.cumsum(pca.explained_variance_ratio_))
plt.axvline(pca_comps, color="red")  
plt.ylim(0.0,1.1)
plt.show()

In [355]:
reduced_data = pca.transform(data)
#print ("Reduced data: ",reduced_data.shape)
kmeans = KMeans(init='k-means++', n_clusters=n_clusters, n_init=10)
kmeans.fit(reduced_data[:,:2])

# Step size of the mesh. Decrease to increase the quality of the VQ.
h = .02     # point in the mesh [x_min, m_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = reduced_data[:, 0].min()-.05, reduced_data[:, 0].max()+.05
y_min, y_max = reduced_data[:, 1].min()-.05, reduced_data[:, 1].max()+.05
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize=(10,10))
#plt.clf()
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap= plt.cm.Pastel2,#cmap=plt.cm.Paired,
           aspect='auto', origin='lower')

#plt.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=4)

#listA=df[df['readmitted'] == 0].index
#plt.plot(reduced_data[listA, 0], reduced_data[listA, 1],'k.', markersize=4, c='g', label='No Readmitted')
#listB=df[df['readmitted'] == 29].index
#plt.plot(reduced_data[listB, 0],reduced_data[listB, 1],'k.', markersize=4, c='r',label ='<30')
#listC=df[df['readmitted'] == 31].index
#plt.plot(reduced_data[listC, 0],reduced_data[listC, 1],'k.', markersize=4, c='b',label ='>30')

listA=df[df['Congestive_heart_failure'] == 0].index
plt.plot(reduced_data[listA, 0], reduced_data[listA, 1],'k.', markersize=4, c='g', label='No disease')
listB=df[df['Congestive_heart_failure'] == 1].index
plt.plot(reduced_data[listB, 0],reduced_data[listB, 1],'k.', markersize=4, c='r',label ='Disease')

#Peripheral_vascular_disease', 'Cerebrovascular_disease', 'Dementia',
#       'Chronic_pulmonary_disease', 'Myocardial_infarction',
#       'Congestive_heart_failure'

# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(centroids[0, 0], centroids[0, 1],
            marker='>', s=169, linewidths=3, 
            color='black', zorder=10)
plt.scatter(centroids[1, 0], centroids[1, 1],
            marker='H', s=169, linewidths=3, 
            color='black', zorder=10)
plt.title('K-means clustering on the diabetes dataset (PCA-reduced data)\n'
          'Centroids are marked with white cross')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.legend()
plt.show()

In [356]:
num_Myocardial_infarction = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    if (data[i][df_3.columns.get_loc("Myocardial_infarction")] == 0):
        num_Myocardial_infarction[cluster_labels[i]] = num_Myocardial_infarction[cluster_labels[i]] + 1
print ("cluster_sizes = ", [ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ])
print ("num_no_diabetes = ", num_Myocardial_infarction)

num_Peripheral_vascular_disease = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    if (data[i][df_3.columns.get_loc("Peripheral_vascular_disease")] == 0):
        num_Peripheral_vascular_disease[cluster_labels[i]] = num_Peripheral_vascular_disease[cluster_labels[i]] + 1
print ("cluster_sizes = ", [ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ])
print ("num_no_Peripheral_vascular_disease = ", num_Peripheral_vascular_disease)

num_Cerebrovascular_disease = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    if (data[i][df_3.columns.get_loc("Cerebrovascular_disease")] == 0):
        num_Cerebrovascular_disease[cluster_labels[i]] = num_Cerebrovascular_disease[cluster_labels[i]] + 1
print ("cluster_sizes = ", [ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ])
print ("num_no_Cerebrovascular_disease = ", num_Cerebrovascular_disease)

num_Dementia = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    if (data[i][df_3.columns.get_loc("Dementia")] == 0):
        num_Dementia[cluster_labels[i]] = num_Dementia[cluster_labels[i]] + 1
print ("cluster_sizes = ", [ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ])
print ("num_no_Dementia = ", num_Dementia)

num_Chronic_pulmonary_disease = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    if (data[i][df_3.columns.get_loc("Chronic_pulmonary_disease")] == 0):
        num_Chronic_pulmonary_disease[cluster_labels[i]] = num_Chronic_pulmonary_disease[cluster_labels[i]] + 1
print ("cluster_sizes = ", [ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ])
print ("num_no_Chronic_pulmonary_disease = ", num_Chronic_pulmonary_disease)

num_Congestive_heart_failure = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    if (data[i][df_3.columns.get_loc("Congestive_heart_failure")] == 0):
        num_Congestive_heart_failure[cluster_labels[i]] = num_Congestive_heart_failure[cluster_labels[i]] + 1
print ("cluster_sizes = ", [ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ])
print ("num_no_Congestive_heart_failure = ", num_Congestive_heart_failure)

num_readmitted = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    if (data[i][df_3.columns.get_loc("readmitted")] == 0):
        num_readmitted[cluster_labels[i]] = num_readmitted[cluster_labels[i]] + 1
print ("cluster_sizes = ", [ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ])
print ("num_no_readmitted = ", num_readmitted)

num_age = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
        num_age[cluster_labels[i]] = num_age[cluster_labels[i]] + df_4[i][df_3.columns.get_loc("age")]
mida_cluster=[ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ]
print ("average age = ", list(map(truediv,num_age,mida_cluster)))

num_age = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
        num_age[cluster_labels[i]] = num_age[cluster_labels[i]] + df_4[i][df_3.columns.get_loc("weight")]
mida_cluster=[ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ]
print ("average weight = ", list(map(truediv,num_age,mida_cluster)))

cluster_sizes =  [37545, 43763]
num_no_diabetes =  [35744, 41736]
cluster_sizes =  [37545, 43763]
num_no_Peripheral_vascular_disease =  [36825, 42947]
cluster_sizes =  [37545, 43763]
num_no_Cerebrovascular_disease =  [35385, 41460]
cluster_sizes =  [37545, 43763]
num_no_Dementia =  [37483, 43663]
cluster_sizes =  [37545, 43763]
num_no_Chronic_pulmonary_disease =  [32129, 38209]
cluster_sizes =  [37545, 43763]
num_no_Congestive_heart_failure =  [31014, 36294]
cluster_sizes =  [37545, 43763]
num_no_readmitted =  [19316, 24473]
average age =  [70.801438274071117, 70.581541484815943]
average weight =  [92.670186504916401, 92.530952454142053]


In [363]:
#Decision tree
print(newdata_test.shape)
print(newdata.shape)
pca_test = PCA().fit(newdata_test)
reduced_data_test = pca_test.transform(newdata_test)
pca = PCA().fit(newdata)
reduced_data = pca_test.transform(newdata)
n_clusters=2
pca_comps=11
kmeans = KMeans(init='k-means++', n_clusters=n_clusters, n_init=10)
kmeans.fit(reduced_data[:,:pca_comps])
y_train=kmeans.labels_
y_test=kmeans.predict(reduced_data_test[:,:pca_comps])
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from sklearn import tree
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import roc_auc_score

#param_grid = {"criterion": ["gini", "entropy"],
#              "min_samples_split": [2, 10, 20],
#              "max_depth": [None, 2, 5, 10],
#              "min_samples_leaf": [1, 5, 10],
#              "max_leaf_nodes": [None, 5, 10, 20],
#              }

param_grid = {"criterion": ["gini"],
              "min_samples_split": [2],
              "max_depth": [None],
              "min_samples_leaf": [5],
              "max_leaf_nodes": [5],
              }

tree2 = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=param_grid)

tree2.fit(reduced_data, y_train)
y_pred = tree2.predict(reduced_data_test)

#max_depth=4
#clf = tree.DecisionTreeClassifier(random_state=13)
#X_train, X_test, y_train, y_test = train_test_split(data, kmeans.labels_, test_size=0.2)
#y_pred = clf.fit(df_4, y_train).predict(df_4_test)
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
class_names=['cluster0','cluster1']
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()
from sklearn.metrics import accuracy_score
print("Accuracy score", accuracy_score(y_test, y_pred))
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=class_names))

print("Best parameters set found on development set:")
print()
print(tree2.best_params_)
    
with open("C://Users/laia.subirats/Documents/output_diabetes.dot", "w") as output_file:
    tree.export_graphviz(tree2.best_estimator_, out_file=output_file, feature_names=df_3.columns.tolist(),class_names=class_names)

(20458, 49)
(81308, 49)
Confusion matrix, without normalization
[[10992     0]
 [    0  9466]]
Normalized confusion matrix
[[ 1.  0.]
 [ 0.  1.]]
Accuracy score 1.0
             precision    recall  f1-score   support

   cluster0       1.00      1.00      1.00     10992
   cluster1       1.00      1.00      1.00      9466

avg / total       1.00      1.00      1.00     20458

Best parameters set found on development set:

{'min_samples_leaf': 5, 'max_leaf_nodes': 5, 'max_depth': None, 'criterion': 'gini', 'min_samples_split': 2}


In [368]:
#P-values of attributes of each cluster
import scipy.stats as stats
#true_mu = 0
listaA=df_3.columns.tolist()
cluster1=df_4[kmeans.labels_==0]
cluster2=df_4[kmeans.labels_==1]
cluster_labels=kmeans.labels_
#print(cluster1.shape)
#print(cluster2.shape)
contador=0
for z in range(0,len(df_3.columns.tolist())):
    two_sample = stats.ttest_ind(cluster1[:,z],cluster2[:,z])
    if(two_sample[1]<0.05):contador=contador+1

w, h = 3, contador
Matrix = [[0.0 for x in range(w)] for y in range(h)] 
i=0
for z in range(0,len(df_3.columns.tolist())):
    #print(df_2.loc[1,z])
    #print(np.std(cluster1[:,z]))
    #print(df_2.values[:,z])
    two_sample = stats.ttest_ind(cluster1[:,z],cluster2[:,z])
    #two_sample = stats.chisquare(cluster1[:,8],cluster2[:,8])
    if(two_sample[1]<0.05): 
        #print(i)
        Matrix[i][0]=listaA[z]
        Matrix[i][1]='{0:.400f}'.format(two_sample[1])
        Matrix[i][2]=two_sample[1]
        i=i+1
arr = np.array(Matrix)
arr = arr[arr[:,1].argsort()]
#print(arr)

for z in range(0,h):
    if (0<1):
        num_hipertensos = [0] * n_clusters
        for i in range(len(data[:,:2])):
            num_hipertensos[cluster_labels[i]] = num_hipertensos[cluster_labels[i]] + df_4[i][df_3.columns.get_loc(arr[z,0])]
        mida_cluster=[ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ]
        print ((arr[z,0]),": ", list(map(truediv,num_hipertensos,mida_cluster)),", p-value: ",arr[z,2])
    else:
        num_hipertensos = [0] * n_clusters
        for i in range(len(data[:,:2])):
            #print("z",z)
            #print("i",i)
            if (data[i][df_3.columns.get_loc(arr[z,0])] == 1):
                num_hipertensos[cluster_labels[i]] = num_hipertensos[cluster_labels[i]] + 1
        mida_cluster=[ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ]
        percentage=list(map(truediv,num_hipertensos,mida_cluster))
        percentage[0]=round(percentage[0],2)
        percentage[1]=round(percentage[1],2)
        print ((arr[z,0]),": ", num_hipertensos,", percentage: ",percentage, "p-value: ",arr[z,2])

C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 101766 but corresponding boolean dimension is 81308
C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 101766 but corresponding boolean dimension is 81308


admission_source_id :  [5.9320887507711992, 5.7227593554401386] , p-value:  3.25032009654e-12
discharge_disposition_id :  [3.9795717843840688, 3.7993874017845251] , p-value:  3.55847344377e-06
change :  [0.44076045974910311, 0.45691836462911173] , p-value:  3.8572986985e-06
glyburide-metformin :  [0.0051641797865777028, 0.0076441603409242243] , p-value:  2.22039940658e-05
num_lab_procedures :  [43.628430409249823, 43.088693567718735] , p-value:  6.50845410439e-05
weight :  [92.530952454142053, 92.670186504916401] , p-value:  8.83913055025e-05
number_emergency :  [0.16413408587162673, 0.18500466107337862] , p-value:  0.000116787176983
glyburide :  [0.10659689692205744, 0.096577440404847517] , p-value:  0.000179015885585
number_diagnoses :  [7.2297145990905562, 7.2772672792648825] , p-value:  0.000590839824494
pioglitazone :  [0.064986404039942422, 0.071194566520175792] , p-value:  0.00128522165858
max_glu_serum :  [7.379452962548271, 6.4221334398721535] , p-value:  0.00211068396078
num_